In [1]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF']='max_split_size_mb:128'
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys

current_file_directory = os.path.dirname(os.path.abspath(''))
absolute_src_path = os.path.abspath(current_file_directory)
sys.path.append(absolute_src_path)

from src.training.training_utils import *

import transformers

transformers.logging.set_verbosity_debug()

In [2]:
# bnb_config = BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_use_double_quant=True,
#         bnb_4bit_quant_type="nf4",
#         bnb_4bit_compute_dtype=torch.bfloat16,
#     )

model = AutoModelForCausalLM.from_pretrained(
        "mistralai/Mixtral-8x7B-Instruct-v0.1", attn_implementation="flash_attention_2",
       # device_map='auto', #quantization_config=bnb_config
    )

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mixtral-8x7B-Instruct-v0.1/snapshots/125c431e2ff41a156b9f9076f744d2f35dd6e67a/config.json
Model config MixtralConfig {
  "_name_or_path": "mistralai/Mixtral-8x7B-Instruct-v0.1",
  "architectures": [
    "MixtralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mixtral",
  "num_attention_heads": 32,
  "num_experts_per_tok": 2,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "num_local_experts": 8,
  "output_router_logits": false,
  "rms_norm_eps": 1e-05,
  "rope_theta": 1000000.0,
  "router_aux_loss_coef": 0.02,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.37.2",
  "use_cache": true,
  "vocab_size": 32000
}

Detecte

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing MixtralForCausalLM.

All the weights of MixtralForCausalLM were initialized from the model checkpoint at mistralai/Mixtral-8x7B-Instruct-v0.1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use MixtralForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mixtral-8x7B-Instruct-v0.1/snapshots/125c431e2ff41a156b9f9076f744d2f35dd6e67a/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2
}



In [4]:
from peft import PeftModelForCausalLM

peft_config = LoraConfig(
        r=32,
        lora_alpha=8,
        target_modules=[
            "gate",
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
            "w1",
            "w2",
            "w3",
        ],
        lora_dropout=0.05,
        task_type="CAUSAL_LM",
    )

lora_model = PeftModelForCausalLM.from_pretrained(model, '/workspace/best-model', config=peft_config)

In [6]:
lora_model.requires_grad_(False)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MixtralForCausalLM(
      (model): MixtralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MixtralDecoderLayer(
            (self_attn): MixtralFlashAttention2(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear(
                (base_layer): Linear(in

In [11]:
lora_model.to(torch.bfloat16).to('cpu')

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MixtralForCausalLM(
      (model): MixtralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MixtralDecoderLayer(
            (self_attn): MixtralFlashAttention2(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear(
                (base_layer): Linear(in

In [13]:
merged_model = lora_model.merge_and_unload()

KeyboardInterrupt: 

In [ ]:
merged_model.save_pretrained('/workspace/aidx-copilot')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('mistralai/Mixtral-8x7B-Instruct-v0.1')
tokenizer.chat_template = "{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"  # restore padding side
tokenizer.init_kwargs["padding_side"] = "left"

tokenizer.save_pretrained('/workspace/aidx-copilot')

In [7]:
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained('mistralai/Mixtral-8x7B-Instruct-v0.1')
tokenizer.chat_template = "{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"  # restore padding side
tokenizer.init_kwargs["padding_side"] = "left"

# generator = pipeline("conversational", model=lora_model, tokenizer=tokenizer)
generator = pipeline("conversational", model=model, tokenizer=tokenizer)


loading file tokenizer.model from cache at /root/.cache/huggingface/hub/models--mistralai--Mixtral-8x7B-Instruct-v0.1/snapshots/125c431e2ff41a156b9f9076f744d2f35dd6e67a/tokenizer.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mixtral-8x7B-Instruct-v0.1/snapshots/125c431e2ff41a156b9f9076f744d2f35dd6e67a/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mixtral-8x7B-Instruct-v0.1/snapshots/125c431e2ff41a156b9f9076f744d2f35dd6e67a/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mixtral-8x7B-Instruct-v0.1/snapshots/125c431e2ff41a156b9f9076f744d2f35dd6e67a/tokenizer_config.json


In [8]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StoppingCriteriaSub(StoppingCriteria):
    def __init__(self, stops = [], encounters=1, tokenizer=None):
        super().__init__()
        self.stops = stops
        self.ENCOUNTERS = encounters
        self.tokenizer = tokenizer

        assert tokenizer is not None, "Tokenizer is required"

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        stop_count = 0
        for input_ids_list in input_ids:
            for stop in self.stops:
                length = len(stop) + 5 # buffer for special tokens preceeding stop
                
                if len(input_ids_list) < length:
                    continue

                last_elements = input_ids_list[-length:]
                decoded_elements = self.tokenizer.decode(last_elements)

                if stop in decoded_elements:
                    stop_count += 1

        if stop_count >= self.ENCOUNTERS:
            return True

        return False

stop_words = ["<|im_end|>"]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words, tokenizer=tokenizer)])

In [9]:
system_msg = 'You are AIDx, a specialized Large Language Model designed to assist medical professionals in clinical decision-making. Your primary task is to analyze patient data and provide comprehensive, medically sound recommendations. Each output you provide should encompass detailed suggestions for potential diseases, personalized treatment options, and recommended future actions. It is imperative that your advice rigorously aligns with the latest medical standards, emphasizing accuracy, relevance, and patient safety. Your role is crucial in supporting healthcare professionals, enabling them to make well-informed, evidence-based decisions regarding patient care. When responding to inquiries, it is essential to demonstrate a clear and thorough thought process. Begin by methodically analyzing and interpreting each symptom, sign, and lab finding presented in the query. Use this analysis to inform and structure your suggestions, ensuring each reasoning step is explicitly articulated and medically justified. This approach will foster deeper understanding and trust in your assessments and recommendations among medical professionals. Due to regulatory restrictions, avoid using the word diagnosis, instead use a phrase relating to predicting disease. Given that your target user is a medical professional, include a phrase emphasizing that the advice is AI-generated and should be followed with discretion and caution, reminding users of the inherent limitations of AI in medical decision-making.'

In [21]:
class ConversationChain:
    def __init__(self, system_prompt='', conversation=[], generator=None, stopping_criteria=[], max_length=32_000):
        self.system_msg = {
            'role': 'system',
            'content': system_prompt
        }
                
        self.conversation = [self.system_msg] + conversation
        
        assert generator is not None, "Generator (pipeline) is required!"
        self.generator = generator
        
        self.stopping_criteria = stopping_criteria

        self.max_length = max_length

    def __call__(self, message):
        new_conversation = self.conversation + [{
            'role': 'user',
            'content': message
        }]
        
        return self._predict(new_conversation)

    def _predict(self, input_convo):
        gen_kwargs = {
            'do_sample': True, 'temperature': 0.85, 'top_k': 30, 'top_p': 0.9, 'typical_p': 0.9, 'repetition_penalty': 1.2,
            'max_length': self.max_length, 'stopping_criteria': self.stopping_criteria, 'pad_token_id': self.generator.tokenizer.eos_token_id
        }
        
        if isinstance(input_convo, str):
            output = self.generator([
                self.system_msg,
                {
                        'role': 'user',
                        'content': input_convo
                }
                ],
                **gen_kwargs
            )
            
        elif isinstance(input_convo, list):
            assert type(input_convo[0]) == dict, "Conversation must be of type List[Dict]"
            
            # transformed_convo = [{'role': k, 'content': v} for d in input_convo for k, v in d.items()]
    
            # final_convo = [self.system_msg] + transformed_convo
            output = self.generator(input_convo, **gen_kwargs)

        self.conversation = list(output)

        return output[-1]['content']

In [ ]:
test = ConversationChain(system_prompt=system_msg, generator=generator, stopping_criteria=stopping_criteria)

test("My patient is presenting with persistent pain in their right knee (about a 7/10). They got this after squatting, but it appeared after they woke up and they said they didn't feel anything when they went to bed. They also mentioned that they have been experiencing some swelling and stiffness in the knee. They have been taking some over-the-counter pain medication, but it hasn't been helping much. They also mentioned that they have been experiencing some clicking and popping in the knee when they move it. They have no history of trauma to the knee, and they have not had any recent illnesses or fevers. They have no other significant medical history.")

In [22]:
%%time
conversation = [
    {
        'role': 'user',
        'content': '''A 49 year(s) old female is here for consultation for joint pain and back pain and referred by PCP. Today patient complains of mild (pain level 3 /10), intermittent, waxing and waning pain in both hands, both hips and lower back for several years. States pain is worsened by activity, movement, sitting too long, standing too long and standing up from the chair, somewhat relieved by Tylenol Arthritis, Aleve PRN, Diclofenac bid (dose unrecalled at this time), heat and medication. Has morning stiffness lasts for about 2-3 hours. She denies any fever, chills, cough and shortness of breath.
    She states she is experiencing chronic low back pain to which she had undergone PT with slight benefit (spinal manipulation) and Chiropractor without benefit. She notes undergoing X-ray of the L-Spine in 2007 ordered by her Chiropractor, which revealed Lumbar DDD. She reports minimal to moderate difficulty transitioning from sitting to standing and getting out of bed due to stiffness of the low back and b/l hips. At worst, her pain can be as high as 5-6/10. In 2022, she mentions being started on Diclofenac bid (dosage unrecalled) with slight benefit.
    
    I want you to give me a detailed analysis of the patient's information, followed by a comprehensive reasoning for your diagnosis and suggested next steps in treatment.''',
    },
    {
        'role': 'assistant',
        'content': '''Patient Information:
- Age: 49 years old
- Gender: Female
- Complaints: Mild, intermittent, waxing and waning pain in both hands, both hips and lower back for several years; chronic low back pain; minimal to moderate difficulty transitioning from sitting to standing and getting out of bed due to stiffness of the low back and bilateral hips

Relevant History:
- Pain worsens with activity, movement, sitting or standing for prolonged periods, and standing up from a seated position
- Morning stiffness lasting for approximately 2-3 hours
- Partial relief with over-the-counter medications such as acetaminophen and ibuprofen, prescription diclofenac, heat, and rest
- Prior physical therapy yielded some improvement but was limited
- Spinal manipulations through chiropractic care did not alleviate symptoms
- Imaging studies in 2007 showed lumbar degenerative disc disease (L-Spine X-ray)
- No reported fevers, chills, cough, or shortness of breath

Potential Diagnoses:
Based on the provided information, the following conditions could account for the patient's complaints:

1. Osteoarthritis (OA): The most common form of arthritis affecting synovial joints characterized by cartilage degradation, osteophyte formation, subchondral bone sclerosis, and cyst formation leading to functional limitation and often accompanied by pain and inflammation. The distribution of painful sites—bilateral hands, hips, and lower back—supports an OA diagnosis, particularly given age and radiographic findings consistent with lumbar spondylosis. Furthermore, her response to nonsteroidal anti-inflammatory drugs (NSAIDs) supports the possibility of OA involvement. However, other forms of arthritis cannot be ruled out based solely on history and examination.

Other Potential Conditions Considered:
While there remains concern for alternative diagnoses, they appear less likely given the history and exam details. Nonetheless, additional possibilities warrant consideration, including rheumatologic causes like psoriatic arthritis, reactive arthritis, septic arthritis, polymyalgia rheumatica (PMR)/ giant cell arteritis (GCA), seronegative enthesopathy syndrome (SES), and crystal arthropathies such as gout and pseudogout. Less probable explanations may also involve neurological disorders mimicking musculoskeletal manifestations, although these scenarios seem substantially more remote.

Recommended Next Steps in Treatment:
1. Continue current NSAID regimen if tolerated, preferably prescribed doses rather than over-the-counter alternatives. Monitor side effects closely, especially considering gastric ulcers risk associated with long-term usage. Alternatively, consider switching to topical NSAIDs or capsaicin cream application locally onto affected areas to minimize systemic absorption. Additionally, explore adding muscle relaxants during flare-ups, e.g., cyclobenzaprine or tizanidine, to augment analgesia while minimizing adverse events compared to opioids.
2. Encourage weight loss strategies, regular aerobic exercise programs, and flexibility training tailored to individual needs and abilities when feasible since improvements in mobility can reduce mechanical stress placed upon vulnerable articular structures implicated in pathogenesis. Consult physiatry specialists familiar with managing complex regional pain syndromes involving multiple peripheral joints concurrently for guidance on appropriate therapeutic exercises addressing specific impairments identified via focused assessment.
3. Refer patients to orthopedics or sports medicine clinicians who specialize in regenerative treatments such as platelet-rich plasma (PRP) injections, viscosupplementation procedures, autologous stem cell therapies, etc., potentially offering substantial benefits beyond traditional conservative measures alone. Ensure proper screening tests have been conducted before proceeding further to rule out underlying bleeding disorders, coagulopathies, active malignancies, or infectious processes posing contraindications towards invasive intervention methods.
4. Initiate collaborative discussions between all relevant providers involved within multifactorial management plans concerning shared goals, expectations, timelines, follow-up appointments, communication channels, available resources, educational materials, and support networks aimed at optimizing overall quality of life outcomes achievable across diverse domains impacted adversely by persistent discomfort.

AI Generated Advice Disclaimer:
The above recommendations were generated by an artificial intelligence model trained to identify various medical conditions and suggest possible courses of action.'''
    }
]

test = ConversationChain(system_prompt=system_msg, conversation=conversation, generator=generator, stopping_criteria=stopping_criteria)

CPU times: user 19 µs, sys: 0 ns, total: 19 µs
Wall time: 26.2 µs


In [23]:
%%time
print(test('''The patient reported being diagnosed with Psoriasis on scalp, face, eyebrows, behind the ears, inside the ears, and b/l LE in 2004 by her Dermatologist treated by topical medications (cream and ointment). Currently, she only notes of having a Plaque at the base of her hairline.'''))

CPU times: user 2min 4s, sys: 16.9 s, total: 2min 21s
Wall time: 2min 22s


'Thank you for providing updated information regarding the patient\'s past dermatological condition. Based on the new input, we must reassess our previous conclusions and consider whether the original hypothesis of osteoarthritis sufficiently explains all symptoms described. With particular attention drawn to the report of psoriasis affecting the scalp, face, eyebrows, behind the ears, inside the ears, and bilateral legs, an expanded perspective is required. Specifically, it raises concerns for a broader category of autoimmune illnesses known collectively as Spondyloarthritides (SpAs), which comprise a heterogeneous group of disorders sharing overlapping genetic susceptibility factors and frequently exhibiting cutaneous and articular abnormalities alike. These entities encompass predominantly axial presentations classified under Axial SpA and primarily peripheral patterns categorized as Peripheral SpA, whereas others fall into mixed phenotype categories. Within this contextual framewor

In [ ]:
user_in = '''History: A 55-year-old male presents with a 2-month history of progressive fatigue, intermittent low-grade fever, and a 10-pound weight loss. He mentions a persistent, dry cough and mild shortness of breath. The patient has a 20-year history of smoking but quit 5 years ago. He denies any recent travels or sick contacts. He works as a carpenter.

Physical Examination: The patient appears mildly distressed. Vital signs are normal except for a mild fever. Lung auscultation reveals faint crackles at the base of the right lung. There's no lymphadenopathy or clubbing. Skin examination is unremarkable.

Lab Findings:

CBC: Mild anemia.
Chest X-Ray: A small, ill-defined opacity in the right lower lobe.
Spirometry: Mild restrictive pattern.
Sputum Culture: No growth.
Tuberculin Skin Test (TST): Negative.
Serum Electrolytes, Renal and Liver Function Tests: Normal.
Differential Diagnosis:

Lung Cancer: Given the history of smoking and lung opacity.
Tuberculosis (TB): Considering the chronic cough and weight loss, despite the negative TST.
Sarcoidosis: Possible, given the lung findings and restrictive lung disease pattern, but no lymphadenopathy or skin findings typical of sarcoidosis are present.
Hypersensitivity Pneumonitis: Occupational exposure could be a risk factor, but the presentation is atypical.
Further Investigations:

CT scan of the chest for a better characterization of the lung lesion.
Bronchoscopy with biopsy if indicated by CT findings.
Serum ACE levels and possibly a gallium scan to evaluate for sarcoidosis.
Detailed occupational history to assess for hypersensitivity pneumonitis risk.

Give me a detailed analysis of this patient's information, and suggest some potential diagnoses, next steps, and future treatment options.
'''

test = ConversationChain(system_prompt=system_msg, generator=generator, stopping_criteria=stopping_criteria)

test(user_in)

In [53]:
class ConversationChain:
    def __init__(self, system_prompt='', conversation=[], generator=None, stopping_criteria=[], max_length=32_000):
        self.system_msg = {
            'role': 'system',
            'content': system_prompt
        }
        
        self.conversation = [system_msg] + conversation
        
        assert generator is not None, "Generator (pipeline) is required!"
        self.generator = generator
        
        self.stopping_criteria = stopping_criteria

        self.max_length = max_length

    def __call__(self, message):
        new_conversation = self.conversation + [{
            'role': 'user',
            'content': message
        }]

        return self._predict(new_conversation)

    def _predict(self, input_convo):
        gen_kwargs = {
            'do_sample': True, 'temperature': 0.85, 'top_k': 30, 'top_p': 0.9, 'typical_p': 0.9, 'repetition_penalty': 1.2,
            'max_length': self.max_length, 'stopping_criteria': self.stopping_criteria, 'pad_token_id': self.generator.tokenizer.eos_token_id
        }
        
        if isinstance(input_convo, str):
            output = self.generator([
                self.system_msg,
                {
                        'role': 'user',
                        'content': input_convo
                }
                ],
                **gen_kwargs
            )
            
        elif isinstance(input_convo, list):
            assert type(input_convo[0]) == dict, "Conversation must be of type List[Dict]"
            
            # transformed_convo = [{'role': k, 'content': v} for d in input_convo for k, v in d.items()]
    
            # final_convo = [self.system_msg] + transformed_convo
            output = self.generator(input_convo, **gen_kwargs)

        self.conversation = list(output)

        return output[-1]['content']

In [ ]:
batch_size = 1
gradient_accumulation_steps = 4

project = "aidx-finetune"
base_model_name = "mixtral"
run_name = (
        base_model_name
        + "-"
        + project
)
output_dir = "./training/" + run_name
cpu_count = os.cpu_count()

training_args = TrainingArguments(
    output_dir=output_dir,
    warmup_steps=10,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    auto_find_batch_size=True,
    gradient_accumulation_steps=gradient_accumulation_steps,
    gradient_checkpointing=True,
    num_train_epochs=2,
    dataloader_num_workers=cpu_count,
    learning_rate=0.0002,
    lr_scheduler_type='cosine',
    bf16=True,
    optim="paged_adamw_8bit",
    logging_steps=1,
    logging_dir="./logs",  # Directory for storing logs
    save_strategy="steps",
    save_steps=5,  # Save checkpoints every 5 steps
    evaluation_strategy="epoch",
    do_eval=True,
    report_to="wandb",  # Comment this out if you don't want to use weights & baises
    run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}",  # Name of the W&B run (optional)
    gradient_checkpointing_kwargs={
        'use_reentrant': False
    },
    # deepspeed="/workspace/zero3.json",
    resume_from_checkpoint=True
)


In [ ]:
lora_config = LoraConfig(
        r=32,
        lora_alpha=16,
        target_modules=[
            "gate",
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
            "w1",
            "w2",
            "w3",
        ],
        lora_dropout=0.05,
        task_type="CAUSAL_LM",
    )
model = get_peft_model(model, lora_config)

In [ ]:
data_collator = create_data_collator(base_model_id)
compute_bertscore = load_bertscore()

trainer = Trainer(
        model=model,
        # train_dataset=train_data,
        # eval_dataset=val_data,
        args=training_args,
        data_collator=data_collator,
        compute_metrics=compute_bertscore,
    )

In [ ]:
def main():
    parquet_dir = '/workspace/data/parquet'
    stream_data = False
    base_model_id = 'mistralai/Mixtral-8x7B-Instruct-v0.1'
    
    cpu_count = os.cpu_count()
    gpu_count = torch.cuda.device_count()
    
    print("Setting up training environment...")
    accelerator = setup_training_env()
    data_collator = create_data_collator(base_model_id)
    accelerator.print("Done setting up training environment...")
    
    accelerator.print(f"Number of CPUs available: {cpu_count}")
    accelerator.print(f"Number of GPUs available: {gpu_count}")
    
    accelerator.print("Loading data...")
    train_data, val_data, train_count, val_count = load_data(parquet_dir, stream=stream_data, cpu_count=cpu_count, accelerator=accelerator)
    accelerator.print("Done loading data...")
    
    accelerator.print("Loading compute_bertscore...")
    compute_bertscore = load_bertscore()
    accelerator.print("Done loading compute_bertscore...")

    max_batch_size = 1
    num_epochs = 1

    # x = train_data.select(range(100))
    
    # def test(a):
    #     return {
    #         'attention_mask': a['attention_mask'][:512],
    #         'input_ids': a['input_ids'][:512]
    #     }
    
    # x = x.map(test)
    
    accelerator.print("Loading model trainer...")
    trainer = load_model_trainer(
            base_model_id,
            compute_bertscore,
            data_collator,
            train_data,
            val_data,
            max_batch_size,
            num_epochs,
            cpu_count//gpu_count,
            accelerator
        )
    
    print_gpu_utilization(accelerator)

    # train_result = trainer.train()

main()

In [ ]:
from accelerate import notebook_launcher

notebook_launcher(main, num_processes=2)